<a href="https://colab.research.google.com/github/joyalshaji135/CNN-Implementation/blob/master/Transfer_Learning_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
from skimage import data
import matplotlib.pyplot as plt
import numpy as np
from skimage.transform import resize
from pandas import DataFrame

In [3]:
import keras
from tensorflow.keras.layers import Input,Conv2D,MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization

In [4]:
from PIL import Image
from skimage import io
X=[]
Y=[]
base_path='/content/drive/MyDrive/artificalinteligents'
source_path=base_path
for child in os.listdir(source_path):
  print(child)
  sub_path = os.path.join(source_path, child)
  bsub_path = os.path.join(base_path, child)
  if os.path.isdir(sub_path):
    for data_file in os.listdir(sub_path):
      Qry = Image.open(os.path.join(sub_path, data_file))
      Qry = Qry.convert("RGB")
      Qry = np.array(Qry.resize((224,224)))
      Qry = Qry.reshape([224,224,3])
      Qry = Qry[:,:,2]
      flist=np.array(Qry)
      X.append(flist)
      Y.append(child)

Covid
PNEUMONIA
NORMAL


In [5]:
images_arr = np.asarray(X)
images_arr = images_arr.astype('float32')
images_arr = images_arr / np.max(images_arr)

In [6]:
from sklearn.preprocessing import LabelBinarizer
labelBinarizer = LabelBinarizer()
y=labelBinarizer.fit_transform(Y)

In [7]:
from tensorflow.keras.applications.vgg16 import VGG16
model = VGG16(weights='imagenet', include_top=False)
X=[]
for i in range(images_arr.shape[0]):
  img= resize(images_arr[i], (224,224,3),anti_aliasing=True)
  features=model.predict(np.array([img]))
  flist=np.array(features.flatten())
  X.append(flist)

1/1 [==============================] - 0s 20ms/step


In [8]:
X=np.asarray(X)
print(X)

[[0.11674228 0.         0.         ... 0.         0.7369872  0.        ]
 [0.07803287 0.         0.         ... 0.         0.547567   0.        ]
 [0.30080026 0.         0.         ... 0.         0.54929477 0.        ]
 ...
 [0.09169614 0.         0.         ... 0.         0.4364449  0.        ]
 [0.17419603 0.         0.         ... 0.         0.48364252 0.        ]
 [0.8166284  0.         0.01085752 ... 0.         0.38630545 0.        ]]


In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Flatten
import random
import tensorflow as tf

In [11]:
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)
dnnModel=Sequential()
dnnModel.add(Dense(1024,activation="relu",input_shape=(25088,)))
dnnModel.add(Dense(512,activation="relu"))
dnnModel.add(Dense(256,activation="relu"))
dnnModel.add(Dense(128,activation="relu"))
dnnModel.add(Dense(3,activation="softmax"))
dnnModel.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1024)              25691136  
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                                 
 dense_2 (Dense)             (None, 256)               131328    
                                                                 
 dense_3 (Dense)             (None, 128)               32896     
                                                                 
 dense_4 (Dense)             (None, 3)                 387       
                                                                 
Total params: 26380547 (100.63 MB)
Trainable params: 26380547 (100.63 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
dnnModel.compile(optimizer='adam',loss="categorical_crossentropy",metrics=["accuracy"])
dnnModel.fit(X_train,y_train,epochs=100,batch_size=128,verbose=1)

Epoch 1/100
6/6 [==============================] - 4s 19ms/step - loss: 3.8480 - accuracy: 0.3869
Epoch 2/100
6/6 [==============================] - 0s 15ms/step - loss: 1.8739 - accuracy: 0.5076
Epoch 3/100
6/6 [==============================] - 0s 14ms/step - loss: 0.5449 - accuracy: 0.8272
Epoch 4/100
6/6 [==============================] - 0s 15ms/step - loss: 0.3304 - accuracy: 0.8226
Epoch 5/100
6/6 [==============================] - 0s 14ms/step - loss: 0.1644 - accuracy: 0.9465
Epoch 6/100
6/6 [==============================] - 0s 15ms/step - loss: 0.1073 - accuracy: 0.9709
Epoch 7/100
6/6 [==============================] - 0s 15ms/step - loss: 0.1209 - accuracy: 0.9450
Epoch 8/100
6/6 [==============================] - 0s 14ms/step - loss: 0.0671 - accuracy: 0.9755
Epoch 9/100
6/6 [==============================] - 0s 14ms/step - loss: 0.0593 - accuracy: 0.9862
Epoch 10/100
6/6 [==============================] - 0s 15ms/step - loss: 0.0470 - accuracy: 0.9862
Epoch 11/100
6/6 [=

In [13]:
testLoss, testAccuracy = dnnModel.evaluate(X_test,y_test)
print("Test Accuracy =", testAccuracy)

6/6 [==============================] - 0s 5ms/step - loss: 0.2071 - accuracy: 0.9573
Test Accuracy = 0.957317054271698
